In [1]:
import pandas as pd
import numpy as np

In [2]:
r_e = 2.817e-15
c = 3e8
n = 6.9e10
e = 1.6e-19
q=n*e
E_e = 150
mc2 = 0.511
gamma = E_e/mc2
pi = 3.1416
C = 171.7
sigma_s = 0.025
gamma

293.5420743639922

In [3]:
A = r_e*r_e*c*q/(8*pi*e*gamma*gamma*gamma*sigma_s*C)
A

6.019796132036648e-20

In [4]:
df_ = pd.read_csv("tp0.txt",delim_whitespace=True)
s = df_['s']
df_ = (s - np.roll(s, 1))[1:]# second row - first row, 1: removes the first row at the end
df_.reset_index(drop = True, inplace=True)
df_
df0 = pd.DataFrame(df_) #Converting into dataframe
df0

,s
0,1.40
1,0.25
2,2.90
3,0.25
4,2.90
...,...
158,2.90
159,0.25
160,2.90
161,0.25


In [5]:
# Load all twiss parameters and momentum aperture
df1=pd.read_csv("tp1.txt",sep="\s+")
df2=pd.read_csv("tp2.txt",sep="\s+")
df3=pd.read_csv("ma2.txt",sep="\s+")

In [6]:
# Merge df1 df2 and df3
df4 = df0.reset_index().merge(df1.reset_index(), left_index=True, right_index=True, how='left')
df5 = df2.reset_index().merge(df3.reset_index(), left_index=True, right_index=True, how='left')
df = df4.reset_index().merge(df5.reset_index(), left_index=True, right_index=True, how='left')

In [7]:
df

,index_x,index_x_x,s,index_y_x,bx,ax,by,ay,sigx1,sigy,gx,index_y,index_x_y,etx,etxp,index_y_y,dPos,dNeg
0,0,0,1.40,0,3.284000,-5.600000e-01,3.284000,-5.600000e-01,0.000007,0.000598,0.400000,0,0,-4.092565e-10,3.024853e-10,0,0.009,-0.007232
1,1,1,0.25,1,3.842075,-1.724942e+00,3.347376,3.125298e-01,0.000008,0.000604,1.034708,1,1,-3.473679e-10,1.955527e-10,1,0.009,-0.007242
2,2,2,2.90,2,22.548640,-4.725596e+00,4.292519,-6.384410e-01,0.000049,0.000684,1.034708,2,2,2.197350e-10,1.955527e-10,2,0.009,-0.007099
3,3,3,0.25,3,23.187460,2.235301e+00,4.983429,-2.194795e+00,0.000050,0.000737,0.258613,3,3,2.597172e-10,1.222789e-10,3,0.009,-0.007231
4,4,4,2.90,4,12.397650,1.485324e+00,27.530180,-5.579946e+00,0.000027,0.001732,0.258613,4,4,6.143260e-10,1.222789e-10,4,0.009,-0.007231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,158,158,2.90,158,23.187460,-2.235301e+00,4.983429,2.194795e+00,0.000050,0.000737,0.258613,158,158,2.597290e-10,-1.222779e-10,158,0.009,-0.006405
159,159,159,0.25,159,22.548640,4.725596e+00,4.292519,6.384410e-01,0.000049,0.000684,1.034708,159,159,2.197465e-10,-1.955553e-10,159,0.009,-0.006273
160,160,160,2.90,160,3.842075,1.724942e+00,3.347376,-3.125298e-01,0.000008,0.000604,1.034708,160,160,-3.473637e-10,-1.955553e-10,160,0.009,-0.006273
161,161,161,0.25,161,3.284000,5.600000e-01,3.284000,5.600000e-01,0.000007,0.000598,0.400000,161,161,-4.092528e-10,-3.024867e-10,161,0.009,-0.006393


In [8]:
sig_d = 6.92E-04
sig_d_sq = sig_d*sig_d

In [9]:
for ind, row in df.iterrows():
    df.loc[ind,"etx_del_sq"] = sig_d_sq * (row["etx"]*row["etx"])
for ind, row in df.iterrows():
    df.loc[ind,"sigx"] = np.sqrt(row["sigx1"] + row["etx_del_sq"])

In [10]:
for ind, row in df.iterrows():
    df.loc[ind,"H(s)"] = (row["gx"]*row["etx"]*row["etx"]) + (2* row["ax"]*row["etx"]*row["etxp"]) + (row["bx"]*row["etxp"]*row["etxp"])

In [11]:
ex = 2.17e-06
sig_d = 6.92E-04
sig_d_sq = sig_d*sig_d
ratio = sig_d_sq / ex
for ind, row in df.iterrows():
    df.loc[ind,"sig_div"] = (ex /row["sigx"])*np.sqrt(1 + (row["H(s)"]*ratio))

In [12]:
gamma_L = 293.54
for ind, row in df.iterrows():
    df.loc[ind,"denom"] = row["sig_div"]*gamma_L
    
for ind, row in df.iterrows():
    df.loc[ind,"x"] = (row["dPos"]/row["denom"])*(row["dPos"]/row["denom"])

In [13]:
# Integrating F(x) and getting array values of integration for all values of x
from scipy.integrate import quad
import math
from math import log
x_values = df["x"].to_numpy()
I1 = np.empty(len(x_values))
def f(u,x):
    return (1/u - 0.5*np.log(1/u) - 1)*np.exp(-x/u)
for i, x in enumerate(x_values):
    I1[i] = quad(f, 0, 1, args=(x))[0]
I1

array([4.50763658, 4.35455548, 2.67929835, 2.65399688, 3.23170253,
       3.21706078, 2.42321855, 2.4268322 , 3.36310329, 3.42563846,
       3.56932075, 3.5970435 , 4.10430817, 4.09782195, 3.85879411,
       3.81774092, 3.60725257, 3.61522426, 4.04293913, 4.77878024,
       5.20138787, 5.17717721, 4.54910079, 4.12149373, 3.65789134,
       3.68520993, 4.08831161, 4.77098923, 5.28415986, 5.28262165,
       5.18720516, 5.08593229, 4.76946194, 4.56510768, 4.29850018,
       4.30889458, 4.35811471, 4.40702128, 4.54978082, 5.00258156,
       5.51710616, 5.51606439, 5.42045697, 5.31897336, 5.00156166,
       4.21184405, 3.97250767, 3.98360731, 4.03244578, 4.06414201,
       4.22281932, 4.21584454, 4.72460496, 4.72460496, 4.21584454,
       4.05415948, 3.654222  , 3.65933701, 4.12257891, 4.59074962,
       5.22387758, 5.26621642, 4.84226367, 4.04732575, 3.62011052,
       3.60336742, 3.8123453 , 3.85879411, 4.09782195, 4.16964399,
       4.05012599, 2.73310271, 2.78604029, 7.14345354, 6.91105

In [16]:
for ind, row in df.iterrows():
    df.loc[ind,"dPos_sq"] = row["dPos"]*row["dPos"]
for ind, row in df.iterrows():
    df.loc[ind,"I2"] = row["sigx"]*row["sigy"]*row["sig_div"]*row["dPos_sq"]

In [15]:
df["I1"] = pd.Series(I1) # Convert numpy array into dataframe column


In [22]:
for ind, row in df.iterrows():
    df.loc[ind,"I"] = (row["I1"] / row["I2"])*row["s"]
    
I_sum = df["I"].sum()
print(I_sum)

5129872828524800.0


In [23]:
pos_trate = A*I_sum
pos_tlifetime = 1/(pos_trate)
pos_lifetime = pos_tlifetime/3600
pos_lifetime

0.8995164640286729

### Touscheck Rate for delta_negative

In [24]:
gamma_L = 293.54
for ind, row in df.iterrows():
    df.loc[ind,"denom"] = row["sig_div"]*gamma_L
    
for ind, row in df.iterrows():
    df.loc[ind,"y"] = (row["dNeg"]/row["denom"])*(row["dNeg"]/row["denom"])

In [25]:
from scipy.integrate import quad
import math
from math import log
y_values = df["y"].to_numpy()
I1y = np.empty(len(y_values))
def f(u,y):
    return (1/u - 0.5*np.log(1/u) - 1)*np.exp(-y/u)
for i, y in enumerate(y_values):
    I1y[i] = quad(f, 0, 1, args=(y))[0]
I1y

array([4.9364437 , 4.77962021, 3.1159072 , 3.05560792, 3.64616036,
       3.63107265, 2.81773049, 2.75493808, 3.70986882, 3.86550375,
       4.01134919, 3.97263699, 4.55272814, 4.47850187, 4.23728464,
       4.20728643, 3.99447322, 4.03889264, 4.47146776, 5.06385889,
       5.48770626, 5.45499257, 4.82188185, 4.71175827, 4.24665758,
       4.24298884, 4.65110233, 5.10022144, 5.61522783, 5.60450606,
       5.49511094, 5.40720447, 5.07436871, 4.42714736, 4.18688017,
       4.19283808, 4.24225605, 4.2907197 , 4.43324455, 4.70629906,
       5.26066932, 5.26063043, 5.16523613, 5.06398865, 4.7476106 ,
       4.44194778, 4.20158953, 4.21581955, 4.27988398, 4.32871158,
       4.45625029, 4.97660005, 5.52369174, 5.55695795, 4.94461009,
       4.54221715, 4.13455696, 4.13718956, 4.60664636, 4.83400078,
       5.46720256, 5.5006771 , 5.07664717, 4.51614755, 4.08841981,
       4.03838811, 4.25591341, 4.34930075, 4.59105493, 4.66360171,
       4.56059002, 3.21844524, 3.24764677, 7.64462187, 7.41198

In [28]:
for ind, row in df.iterrows():
    df.loc[ind,"dNeg_sq"] = row["dNeg"]*row["dNeg"]
for ind, row in df.iterrows():
    df.loc[ind,"I2y"] = row["sigx"]*row["sigy"]*row["sig_div"]*row["dNeg_sq"]

In [29]:
df["I1y"] = pd.Series(I1y) # Convert numpy array into dataframe column

In [30]:
for ind, row in df.iterrows():
    df.loc[ind,"Iy"] = (row["I1y"]/ row["I2y"])*row["s"] # total integration = numerator divide by denominator

In [31]:
Iy_sum = df["Iy"].sum()
print(Iy_sum)

8108106166832766.0


In [32]:
neg_trate = A*Iy_sum
neg_tlifetime = 1/(neg_trate)
neg_lifetime = neg_tlifetime/3600
neg_lifetime

0.5691100945997972

In [33]:
t_rate = (1/(2*pos_lifetime))+(1/(2*neg_lifetime))
t_rate

1.4344188255402455

In [34]:
tlifetime = 1/t_rate
tlifetime

0.6971464555502956